In [1]:
#Preprocessed data - removed noise. Takes about 2 minutes
import re                                      #imported python regex
from nltk.corpus import stopwords              #imported nltk module, used for preprocessing text data. has about 200 words - a, an, or, the, etc.
from nltk.tokenize import word_tokenize        #import tokenize method - assigns a number to each word, which is a token.
from bs4 import BeautifulSoup                  #It is a html and xml parser, used for scraping web pages.
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
import pandas as pd

#To preprocess dataset
def preprocess_dataset(reviews):
    result = []
    for review in reviews:
        review_text = BeautifulSoup(review, "html.parser").get_text()
        review_text = re.sub(r'[^a-zA-Z]', ' ', review_text)
        review_text = review_text.lower()
        words = word_tokenize(review_text)
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]
        preprocessed_review = ' '.join(words)
        result.append(preprocessed_review)
    return(result)

#Starts here... load the data - create a list of all the reviews.
fp = 'C:\\ml\\data\\IMDBCentiment\\imdb_reviews.csv'
reviews_df = pd.read_csv(fp)
print(reviews_df.shape)
reviews = reviews_df['review'].tolist()
sentiment = reviews_df['sentiment'].tolist()
#preprocess reviews - remove html tags, stopwords, convert all letters to lowercase
result = preprocess_dataset(reviews)


(50000, 2)


C:\Users\mail2\AppData\Local\Temp\ipykernel_25428\2557198865.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review, "html.parser").get_text()


In [2]:
#created features from the reviews in using TF-IDF. - takes about 30 seconds.
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer with desired parameters
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', lowercase=True)

# Fit and transform the preprocessed reviews to create the TF-IDF feature matrix
tfidf_features = tfidf_vectorizer.fit_transform(result)

# Convert the TF-IDF feature matrix to an array
tfidf_features_array = tfidf_features.toarray()

# Print the shape of the TF-IDF feature matrix
print("Shape of TF-IDF feature matrix:", tfidf_features_array.shape)


Shape of TF-IDF feature matrix: (50000, 5000)


In [3]:
#created test and train data for Naive Bayes model. Takes about 30 seconds.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_features_array, sentiment, test_size=0.2, random_state=42)

In [6]:
#Support Vector Machine - Did the same as above with this model. SVM tries to find an optimal plane/line that separates the classes in a dataset.
#support vectors are vectors that lie on the decision boundary. SVMs are used widely for classification, regression, and outlier detection tasks.
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Create an SVM classifier
svm_model = SVC(kernel='linear', random_state=42)
svm_model = SVC(verbose=True)
# Train the SVM model
print("enabled verbose logging")
# Predict labels for the test set
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

enabled verbose logging
